# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234536296272                   -0.50    8.0         
  2   -7.250177372572       -1.81       -1.39    1.0   12.3ms
  3   -7.251077622062       -3.05       -2.05    1.0   12.1ms
  4   -7.251105796805       -4.55       -1.97    2.0   15.9ms
  5   -7.251329994430       -3.65       -2.63    1.0   12.4ms
  6   -7.251338137575       -5.09       -3.18    1.0   12.6ms
  7   -7.251338633047       -6.30       -3.46    2.0   14.4ms
  8   -7.251338785911       -6.82       -3.94    2.0   15.8ms
  9   -7.251338796191       -7.99       -4.38    1.0   13.3ms
 10   -7.251338798559       -8.63       -5.13    2.0   15.4ms
 11   -7.251338798692       -9.88       -5.64    2.0   17.1ms
 12   -7.251338798702      -10.99       -5.85    3.0   17.3ms
 13   -7.251338798704      -11.62       -6.42    1.0   12.8ms
 14   -7.251338798704      -12.66       -6.53    3.0   17.9ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04519429135012608
[ Info: Arnoldi iteration step 2: normres = 0.4545509676827249
[ Info: Arnoldi iteration step 3: normres = 0.785746650277408
[ Info: Arnoldi iteration step 4: normres = 0.47786682730974817
[ Info: Arnoldi iteration step 5: normres = 0.5160638879312818
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.07e-01, 7.99e-02, 3.99e-01, 2.97e-01, 3.30e-02)
[ Info: Arnoldi iteration step 6: normres = 0.329636584622599
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.99e-02, 1.39e-01, 2.59e-01, 1.15e-01, 6.65e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07816922874559946
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.16e-03, 1.24e-02, 1.58e-02, 4.10e-02, 5.69e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10526058960791121
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.27e-05